In [ ]:
import pandas as pd
from uszipcode import SearchEngine

In [ ]:
search = SearchEngine(simple_zipcode=False) # Gives a lot of interesting data if set to False
slalom_latitude = 33.854473
slalom_longitude = -84.360729

closest_zip_codes = search.by_coordinates(slalom_latitude, slalom_longitude, radius=10, returns=1000)
closest_zip_codes = list(map(lambda x: x.zipcode, closest_zip_codes))

print(len(closest_zip_codes))

In [ ]:
df_2018 = pd.read_csv('Data/2018-12.csv')

In [ ]:
# Clean up zip codes
df_2018['ZIP_CD'] = df_2018['ZIP_CD'].astype(str).replace('\.0', '', regex=True)

closest = df_2018[df_2018['ZIP_CD'].isin(closest_zip_codes)]
'''
Want Active Providers PGM_TRMNTN_CD == '0'
'''
closest[['FAC_NAME', 'PRVDR_CTGRY_CD','PGM_TRMNTN_CD', 'TRMNTN_EXPRTN_DT', 'CRTFD_BED_CNT']]

